针对第4周作业：准确率提升到98%

In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 载入数据集
mnist = input_data.read_data_sets(os.path.join(os.getcwd(), 'MNIST_data'), one_hot=True)

Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# 定义批次大小
batch_size = 100
#  计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

In [5]:
# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [6]:
# 定义Dropout的保留比例占位符
keep_prob = tf.placeholder(tf.float32)

In [7]:
# 定义一个lr的变量，初始值为0.01
lr = tf.Variable(0.001, dtype=tf.float32)

In [9]:
# 创建一个简单的神经网络
W1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1))
b1 = tf.Variable(tf.zeros([500]) + 0.1)
L1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
L1_drop = tf.nn.dropout(L1, keep_prob)

In [10]:
W2 = tf.Variable(tf.truncated_normal([500, 300], stddev=0.1))
b2 = tf.Variable(tf.zeros([300]) + 0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop, W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

In [11]:
W3 = tf.Variable(tf.truncated_normal([300, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]) + 0.1)
prediction = tf.nn.softmax(tf.matmul(L2_drop, W3) + b3)

In [15]:
# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

In [16]:
# 训练
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

In [17]:
# 初始化变量
init = tf.global_variables_initializer()

In [18]:
# 结果存放在一个布尔型列表中
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(prediction,1)), tf.float32))

In [21]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(50):
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y: batch_ys, keep_prob:1.0})
        
        learning_rate = sess.run(lr)
        test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y: mnist.test.labels, keep_prob:1.0})
        train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images, y: mnist.train.labels, keep_prob:1.0})
        
        print('Learning Rate: '+ str(learning_rate)+", Iter: " + str(epoch) + ",Testing Accuracy: " + str(test_acc) +',Training Accuracy:' + str(train_acc))



Learning Rate: 0.001, Iter: 0,Testing Accuracy: 0.9516,Training Accuracy:0.9572727
Learning Rate: 0.00095, Iter: 1,Testing Accuracy: 0.9621,Training Accuracy:0.9692364
Learning Rate: 0.0009025, Iter: 2,Testing Accuracy: 0.9666,Training Accuracy:0.9748909
Learning Rate: 0.000857375, Iter: 3,Testing Accuracy: 0.9718,Training Accuracy:0.98147273
Learning Rate: 0.00081450626, Iter: 4,Testing Accuracy: 0.9735,Training Accuracy:0.9859818
Learning Rate: 0.0007737809, Iter: 5,Testing Accuracy: 0.9721,Training Accuracy:0.9876909
Learning Rate: 0.0007350919, Iter: 6,Testing Accuracy: 0.9751,Training Accuracy:0.9886909
Learning Rate: 0.0006983373, Iter: 7,Testing Accuracy: 0.9777,Training Accuracy:0.99112725
Learning Rate: 0.0006634204, Iter: 8,Testing Accuracy: 0.9749,Training Accuracy:0.9903273
Learning Rate: 0.0006302494, Iter: 9,Testing Accuracy: 0.9758,Training Accuracy:0.99241817
Learning Rate: 0.0005987369, Iter: 10,Testing Accuracy: 0.9774,Training Accuracy:0.9939091
Learning Rate: 0.0005

---

tensorboard 网络结构复习

In [2]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# 载入数据集
mnist = input_data.read_data_sets(os.path.join(os.getcwd(), 'MNIst_data'), one_hot=True)

Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIst_data/train-images-idx3-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIst_data/train-labels-idx1-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIst_data/t10k-images-idx3-ubyte.gz
Extracting /Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 05/MNIst_data/t10k-labels-idx1-ubyte.gz


In [5]:
# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

In [6]:
# 为了可视化神经网络结构
# 先要定义一个命名空间
with tf.name_scope('input'):
    # 定义两个占位符
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')
    

In [7]:
# 在定义layer层的命名空间
with tf.name_scope('layer'):
    
    # 在命名空间layer下面还可以再定义 命名空间。
    with tf.name_scope('Weights'):
        W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1), name='W')
    with tf.name_scope('Biases'):
        b = tf.Variable(tf.zeros([10])+0.1)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x, W)  + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
        

In [8]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

In [9]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

In [10]:
init = tf.global_variables_initializer()

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    # 存放图结构
    writer = tf.summary.FileWriter('logs/', sess.graph)

    for epoch in range(1):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
        
        acc = sess.run(accuracy, feed_dict ={x:mnist.test.images, y: mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))



Iter 0,Testing Accuracy 0.8112
